# DScover 2022-2 Group C: AirBNB Crawler
* optimized for python 3.8, selenium 4.1.0
* Initial Commit: 2022.11.22

## 패키지 설치
* 웹드라이버 실행 방법: 구글 크롬 설치 --> 크롬드라이버 다운로드 --> path에 드라이버 경로 지정

In [64]:
#셀레늄 설치: 처음 설치할 때만 주석 처리 지우고 설치해주세요
#!pip install selenium==4.1.0

In [ ]:
#4.1.0 버전 출력되는지 확인하기
print(selenium.__version__)

In [21]:
import selenium
from selenium import webdriver
import time
path='C:\\Users\\ihoon\\Documents\\pyd\\chromedriver_win32\\chromedriver.exe' #크롬 드라이버 경로 지정해주세요
import re
import pandas as pd
import numpy as np

In [22]:
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.options import Options

In [23]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## 수집 준비
url_list에는 수집할 도시의 이름을 key로, 그 도시+아래 검색조건으로 반환되는 url을 value로 지정합니다.
* 검색 조건: 유연한 일정 - 12월 및 1월의 주말

page_no에는 총 몇 페이지를 볼 것인지 지정합니다.

**실제 테스트용 value 먼저 정상 작동 확인한 후 수집용 value 주석처리 해제해서 돌리는 게 좋습니다.**

get_housing_type 및 get_information 함수를 모두 선언해 주세요.


In [63]:

#실제 수집용 값: 분석 목표 데이터셋. 반드시 테스트 후 주석 처리 해제하고 실행해주세요
url_list={
          #'마포구': 'https://www.airbnb.co.kr/s/%EB%A7%88%ED%8F%AC%EA%B5%AC--%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C--%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&price_filter_input_type=0&price_filter_num_nights=2&query=%EB%A7%88%ED%8F%AC%EA%B5%AC%2C%20%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C&place_id=ChIJIWguayCZfDURBR0f-llT68I&date_picker_type=flexible_dates&flexible_trip_dates%5B%5D=january&flexible_trip_dates%5B%5D=december&flexible_trip_lengths%5B%5D=weekend_trip&source=structured_search_input_header&search_type=filter_change',
          #'중구': 'https://www.airbnb.co.kr/s/%EC%A4%91%EA%B5%AC--%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&price_filter_input_type=0&price_filter_num_nights=2&date_picker_type=flexible_dates&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=january&flexible_trip_lengths%5B%5D=weekend_trip&source=structured_search_input_header&search_type=search_query', #중구
          #'종로구': 'https://www.airbnb.co.kr/s/%EC%A2%85%EB%A1%9C%EA%B5%AC--%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C--%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&price_filter_input_type=0&price_filter_num_nights=2&date_picker_type=flexible_dates&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=january&flexible_trip_lengths%5B%5D=weekend_trip&source=structured_search_input_header&search_type=autocomplete_click&query=%EC%A2%85%EB%A1%9C%EA%B5%AC%2C%20%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C&place_id=ChIJS5wn3S-9fDUR5ly4ei6px1A',
          '강남구': 'https://www.airbnb.co.kr/s/%EA%B0%95%EB%82%A8%EA%B5%AC--%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C--%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&price_filter_input_type=0&price_filter_num_nights=2&date_picker_type=flexible_dates&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=january&flexible_trip_lengths%5B%5D=weekend_trip&source=structured_search_input_header&search_type=autocomplete_click&query=%EA%B0%95%EB%82%A8%EA%B5%AC%2C%20%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C&place_id=ChIJ-4m1XyOkfDURartwxRuXMbM',
          #'서대문구': 'https://www.airbnb.co.kr/s/%EC%84%9C%EB%8C%80%EB%AC%B8%EA%B5%AC--%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C--%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&price_filter_input_type=0&price_filter_num_nights=2&date_picker_type=flexible_dates&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=january&flexible_trip_lengths%5B%5D=weekend_trip&source=structured_search_input_header&search_type=autocomplete_click&query=%EC%84%9C%EB%8C%80%EB%AC%B8%EA%B5%AC%2C%20%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C&place_id=ChIJnQRRjmiYfDURXNqeJBtzWRE',
          #'용산구': 'https://www.airbnb.co.kr/s/%EC%9A%A9%EC%82%B0%EA%B5%AC--%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C--%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&price_filter_input_type=0&price_filter_num_nights=2&date_picker_type=flexible_dates&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=january&flexible_trip_lengths%5B%5D=weekend_trip&source=structured_search_input_header&search_type=autocomplete_click&query=%EC%9A%A9%EC%82%B0%EA%B5%AC%2C%20%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C&place_id=ChIJ0z8xfjyifDURF7H5KqUsNKQ'
          }
page_no=[6]#12, 7, 7, 6, 5, 5]

'''
#테스트용 값: 코드 정상 작동 확인용. 확인 후 삭제 혹은 주석처리해주세요.
url_list={'영등포구':'https://www.airbnb.co.kr/s/%EC%98%81%EB%93%B1%ED%8F%AC%EA%B5%AC--%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C--%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&price_filter_input_type=0&price_filter_num_nights=2&query=%EC%98%81%EB%93%B1%ED%8F%AC%EA%B5%AC%2C%20%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C&date_picker_type=flexible_dates&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=january&source=structured_search_input_header&search_type=autocomplete_click&place_id=ChIJk_FAvx2ffDURQAuECsjy0Cw&flexible_trip_lengths%5B%5D=weekend_trip',
          '중랑구':'https://www.airbnb.co.kr/s/%EC%A4%91%EB%9E%91%EA%B5%AC--%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C--%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&price_filter_input_type=0&price_filter_num_nights=2&query=%EC%A4%91%EB%9E%91%EA%B5%AC%2C%20%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C&date_picker_type=flexible_dates&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=january&source=structured_search_input_header&search_type=autocomplete_click&flexible_trip_lengths%5B%5D=weekend_trip&place_id=ChIJWUd0flq6fDURt0PJ1_ZNip0'
         } #수집 url (dict)
page_no=[2, 2] #수집 
'''

"\n#테스트용 값: 코드 정상 작동 확인용. 확인 후 삭제 혹은 주석처리해주세요.\nurl_list={'영등포구':'https://www.airbnb.co.kr/s/%EC%98%81%EB%93%B1%ED%8F%AC%EA%B5%AC--%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C--%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&price_filter_input_type=0&price_filter_num_nights=2&query=%EC%98%81%EB%93%B1%ED%8F%AC%EA%B5%AC%2C%20%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C&date_picker_type=flexible_dates&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=january&source=structured_search_input_header&search_type=autocomplete_click&place_id=ChIJk_FAvx2ffDURQAuECsjy0Cw&flexible_trip_lengths%5B%5D=weekend_trip',\n          '중랑구':'https://www.airbnb.co.kr/s/%EC%A4%91%EB%9E%91%EA%B5%AC--%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C--%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&price_filter_input_type=0&price_filter_num_nights=2&query=%EC%A4%91%EB%9E%91%EA%B5%AC%2C%20%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B

In [64]:
#housing type 반환 함수
def get_housing_type(name):
    '''숙소 이름 입력 --> 숙소 유형 반환 함수'''
    housing_name=['전체$','아파트$', '집$','하우스$','주택$','개인실$', '객실$', '다인실$']
    i=0
    try:
        while True:
            match_result=re.search(housing_name[i], name)
            if match_result!=None:
                break
            else:
                i+=1
        if i<=4:
            t_housing_type=1 #'entire_home'
        elif i<=6:
            t_housing_type=2 #'private room'
        elif i==7:
            t_housing_type=3 #'shared room'
        else:
            t_housing_type='NA'
    except:
        t_housing_type=1
    return t_housing_type

In [65]:
#내부 정보 크롤링
def get_information(driver):
    global price
    global rating
    global nrat
    global sh
    global title
    global housing_type
    global cap
    global bed
    global room
    global bath
    global res_rate
    global cert
    global clean
    global freeparking
    global laundry
    global camera
    global review_list
    global npho
    global location

    #번역 창 닫기
    try:
            driver.find_element_by_xpath('/html/body/div[9]/section/div/div/div[2]/div/div[1]/button').click() #번역 창 닫기
            driver.implicitly_wait(1)
    except NoSuchElementException:
        try:
            webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()
        except:
            dummy=0
    
    #가격
    try:
        price=int(driver.find_element_by_xpath('//*[@id="site-content"]/div/div[1]/div[3]/div/div[2]/div/div/div[1]/div/div/div/div/div/div/div[2]/div/section/div[2]/div').text.replace('총 합계\n₩','').replace(',',''))
    except:
        price='NA'
    
    #평점
    try:    
        rating=float(driver.find_element_by_xpath('//*[@id="site-content"]/div/div[1]/div[1]/div[1]/div/div/div/div/section/div[2]/div[1]/span[1]/span[2]').text.replace(' ·',''))
    except:
        rating=3
    
    #평점 수
    try:
        nrat=int(driver.find_element_by_xpath('//*[@id="site-content"]/div/div[1]/div[4]/div/div/div/div[2]/section/div[1]/span[2]/h2/div/span').text.split(sep=' · ')[-1].replace('후기 ','').replace('개',''))
    except:
        nrat='NA'
    
    #슈퍼호스트 여부
    try:
        sh_text=driver.find_element_by_xpath('//*[@id="site-content"]/div/div[1]/div[1]/div[1]/div/div/div/div/section/div[2]/div[1]/span[3]/span[2]').text
        sh=1
    except NoSuchElementException:
        sh=0
    
    #숙소 이름
    title = driver.find_element_by_xpath('//*[@id="site-content"]/div/div[1]/div[1]/div[1]/div/div/div/div/section/div[1]/span').text
    
    #숙소 유형
    housing_name=driver.find_element_by_xpath('//*[@id="site-content"]/div/div[1]/div[3]/div/div[1]/div/div[1]/div/div/section/div/div/div/div[1]/div/h2').text
    housing_type=get_housing_type(housing_name)
    
    
    #숙박인원/침대/침실/욕실
    amen_list=driver.find_element_by_xpath('//*[@id="site-content"]/div/div[1]/div[3]/div/div[1]/div/div[1]/div/div/section/div/div/div/div[1]/ol').text.split(sep=' · ')
    try:
        cap=int(amen_list[0].replace('최대 인원 ','').replace('명',''))
    except ValueError:
        cap='NA'

    if amen_list[1]=='원룸':
        room=0
    else:
        room=float(amen_list[1].replace('침실 ','').replace('개',''))

    try:
        for i in range(len(amen_list)):
            if re.search('침대',amen_list[i])==None:
                bed=0 if room==0 else 1
            else:
                bed=float(amen_list[i].replace('침대 ','').replace('개',''))
    except:
        bed='NA'

    try:
        for i in range(len(amen_list)):
            if re.search('욕실',amen_list[i])==None:
                bath=0
            elif amen_list[i]=='간이 욕실' :
                    bath=1
            else:
                    bath=float(amen_list[i].replace('단독 사용 ','').replace('공동 사용 ','').replace('욕실 ','').replace('개',''))
    except:
        bath='NA'
                
    
    #응답률
    try:
        res_rate=int(driver.find_element_by_xpath('//*[@id="site-content"]/div/div[1]/div[6]/div/div/div/div[2]/div/section/div[2]/div[2]/ul/li[1]/span').text.replace('%',''))
    except:
        try:
            res_rate=int(driver.find_element_by_xpath('//*[@id="site-content"]/div/div[1]/div[6]/div/div/div/div[2]/div/section/div[2]/div[2]/ul/li[2]/span').text.replace('%',''))
        except:
            try:
                res_rate=int(driver.find_element_by_xpath('//*[@id="site-content"]/div/div[1]/div[6]/div/div/div/div[2]/div/section/div[2]/div/ul/li[1]/span').text.replace('%',''))
            except:
                res_rate='NA'
    
    #본인인증 여부
    try:
        driver.find_element_by_xpath('//*[@id="site-content"]/div/div[1]/div[6]/div/div/div/div[2]/div/section/div[2]/div[1]/div[1]/ul/li[2]/div').text
        cert=1
    except NoSuchElementException:
        cert=0
        
    #청소비
    try:
        prices=driver.find_element_by_xpath('//*[@id="site-content"]/div/div[1]/div[3]/div/div[2]/div/div/div[1]/div/div/div/div/div/div/div[2]/div/section/div[1]').text.split(sep='\n')
        clean=1 if '청소비' in prices else 0
    except NoSuchElementException:
        clean='NA'
    
    
    #편의시설 (보안카메라, 세탁기, 주차)
    driver.implicitly_wait(2)
    try:
        element = driver.find_element_by_xpath('//*[@id="site-content"]/div/div[1]/div[3]/div/div[1]/div/div[6]/div/div[2]/section/div[2]/div/h2') #편의시설
        driver.execute_script("arguments[0].scrollIntoView();", element)
        driver.implicitly_wait(2)
    except:
        dummy=0
    try:
        driver.find_element_by_xpath('//*[@id="site-content"]/div/div[1]/div[3]/div/div[1]/div/div[7]/div/div[2]/section/div[4]/button').click() #시설 더보기
    except NoSuchElementException:
        try:
            driver.find_element_by_xpath('//*[@id="site-content"]/div/div[1]/div[3]/div/div[1]/div/div[6]/div/div[2]/section/div[4]/button').click()
        except NoSuchElementException:
            try:
                driver.find_element_by_xpath('//*[@id="site-content"]/div/div[1]/div[3]/div/div[1]/div/div[5]/div/div[2]/section/div[4]/button').click()
            except:
                try:
                    driver.find_element_by_xpath('//*[@id="site-content"]/div/div[1]/div[3]/div/div[1]/div/div[4]/div/div[2]/section/div[4]/button').click()
                except:
                    try:
                        driver.find_element_by_xpath('//*[@id="site-content"]/div/div[1]/div[3]/div/div[1]/div/div[3]/div/div[2]/section/div[4]/button').click()
                    except:
                        try:
                            driver.find_element_by_xpath('//*[@id="site-content"]/div/div[1]/div[3]/div/div[1]/div/div[2]/div/div[2]/section/div[4]/button').click()
                        except:
                            try:
                                driver.find_element_by_xpath('//*[@id="site-content"]/div/div[1]/div[3]/div/div[1]/div/div[1]/div/div[2]/section/div[4]/button').click()
                            except:
                                dummy=0
    driver.implicitly_wait(2)
    amen_text=driver.find_element_by_xpath('/html/body/div[9]/section/div/div/div[2]/div/div[3]').text #편의시설 텍스트화
    amen_list=amen_text.split(sep='\n')
    freeparking=1 if '건물 내 무료 주차' in amen_list else 0
    laundry=1 if ('세탁기' in amen_list) & ('이용 불가: 세탁기' not in amen_list) else 0
    camera=1 if ('숙소 내 보안 카메라' in amen_list) & ('이용 불가: 숙소 내 보안 카메라' not in amen_list) else 0
    driver.implicitly_wait(3)
    try:
        driver.find_element_by_xpath('/html/body/div[9]/section/div/div/div[2]/div/div[1]/button/span').click() #시설 더보기 닫기
    except:
        try:
            element = driver.find_element_by_xpath('/html/body/div[9]/section/div/div/div[2]/div/div[1]/button/span').click()
            webdriver.ActionChains(driver).move_to_element(element ).click(element ).perform()
        except:
            element = driver.find_element_by_xpath('/html/body/div[9]/section/div/div/div[2]/div/div[1]/button/span').click()
            driver.execute_script("arguments[0].click();", element)
    driver.implicitly_wait(2)
    
    #리뷰 가져오기: 스크롤 없이 한 번에 로드되는 만큼만
    review_list=[]
    try: #리뷰 개수 7개 이상
        driver.find_element_by_xpath('//*[@id="site-content"]/div/div[1]/div[4]/div/div/div/div[2]/section/div[4]/button').click() #리뷰 더보기
        driver.implicitly_wait(2)
        element = driver.find_element_by_xpath('/html/body/div[9]/section/div/div/div[2]/div/div[1]') #리뷰 더보기 닫기
        webdriver.ActionChains(driver).move_to_element(element ).click(element ).perform()
        driver.implicitly_wait(2)
        element = driver.find_element_by_xpath('//*[@id="site-content"]/div/div[1]/div[4]/div/div/div/div[2]/section/div[4]/button') #리뷰 더보기 다시
        webdriver.ActionChains(driver).move_to_element(element ).click(element ).perform()
        driver.implicitly_wait(2)
        for i in range(15):
            try:
                review_text=driver.find_element_by_xpath('/html/body/div[9]/section/div/div/div[2]/div/div[3]/div/div/div/div/section/div/div[2]/div[2]/div['+str(i+1)+']/div[2]').text.replace('\n',' ')
                review_list.append(review_text)
            except:
                break           
    except: #리뷰 개수 6개 이하
        for i in range(6):
            try:
                review_text=driver.find_element_by_xpath('//*[@id="site-content"]/div/div[1]/div[4]/div/div/div/div[2]/section/div[3]/div/div/div['+str(i+1)+']/div/div[2]').text
                review_list.append(review_text)
            except:
                continue
    driver.implicitly_wait(1)
    try:
        driver.find_element_by_xpath('/html/body/div[9]/section/div/div/div[2]/div/div[1]/button').click()
    except:
        try:
            webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()
        except:
            dummy=0
    driver.implicitly_wait(7)
    
    #위치 정보(위도, 경도)
    element = driver.find_element_by_xpath('//*[@id="site-content"]/div/div[1]/div[5]/div/div/div/div[2]/div/section/div[1]') #지도
    driver.execute_script("arguments[0].scrollIntoView();", element)
    driver.implicitly_wait(2)
    try:
        driver.find_element_by_xpath('//*[@id="site-content"]/div/div[1]/div[5]/div/div/div/div[2]/div/section/div[3]/div[4]/div/div/div/div[14]/div/a').click()
        p = driver.current_window_handle
        chwd = driver.window_handles
        for w in chwd:
            if(w!=p):
                driver.switch_to.window(w)
        map_url=driver.current_url
        location=map_url.split(sep='&')[0].replace('https://www.google.com/maps?ll=','').split(sep=',')
        driver.close() #지도 탭 닫기
        driver.switch_to.window(p) #개별 숙소 페이지로 돌아옴
    except:
        location='NA'
        
    driver.set_window_size(510,1080) #창 크기 변경
    try:
        npho=int(driver.find_element_by_xpath('//*[@id="site-content"]/div/div[1]/div[2]/div/div/div/div/div/span').text.replace('전체 ','').replace('장 중 1번째 사진 표시',''))
    except:
        npho='NA'
    driver.maximize_window() #다시 창 최대화

    #아직 구현 못한 것들
        #감성분석값: review_score
    
    return price, rating, nrat, sh, title, housing_type, cap, bed, room, bath, res_rate, cert, clean, freeparking, laundry, camera,review_list, npho, location

## 스크래핑 코드
line 24의 range()안의 값은 **한 페이지 안에서 몇 개의 숙소를 가져올 지**를 의미하는데, 수집 시에는 20을 사용해야 합니다.

실제 수집 전 테스트용으로 돌려 볼 때는 값을 '2' 정도로 줄여서 돌려 보는 게 좋습니다.

현재 확인된 문제점
* 처음 수집하는 숙소의 가격, 리뷰 수, 응답률이 반환되지 않는 문제. 수동으로 결측치 처리 필요
* 수집에 1분 이상 걸리는 숙소가 일부 존재. 에러 토하지 않는 한 그냥 두면 돌아감

In [66]:
driver=webdriver.Chrome(path) #inititialize
driver.implicitly_wait(2)
driver.maximize_window()

houseinfo=pd.DataFrame(columns=
                       ['gu','price', 'rating', 'nrat', 'sh', 'title', 'housing_type',\
                        'cap', 'bed', 'room', 'bath', 'res_rate', 'cert', 'clean', 'freeparking',\
                        'laundry', 'camera','review_list', 'npho','location','url'])

for i in range(len(url_list)): #구 단위 반복
    print('\n\n=============================\n{} 숙소 정보 수집 시작\n=============================\n'.format(list(url_list.keys())[i]))
    url=(list(url_list.values())[i])
    gu=list(url_list.keys())[i]
    driver.get(url)
    time.sleep(2)
    wait=WebDriverWait(driver,4) #1페이지 로드 완료
    
    for j in range(int(page_no[i-1])): #페이지 단위 반복
        print('\n{}/{}번째 페이지 로드'.format(j+1,int(page_no[i])))
        original_window=driver.current_window_handle #목록 페이지
    
    
        #i페이지에서 개별 숙소 정보 수집
        for k in range(20): #개별 숙소마다 반복. 한 페이지에 숙소 몇 개 수집할 지
            driver.find_element_by_xpath('//*[@id="site-content"]/div[2]/div[3]/div/div/div/div/div/div['+str(k+1)+']/div/div[2]/div/div/div/div[1]/div').click() #숙소 클릭
            wait.until(EC.number_of_windows_to_be(2)) #창 두개 뜰 때까지 대기
            for window_handle in driver.window_handles: #Loop until finding a new window handle
                if window_handle!=original_window:
                    driver.switch_to.window(window_handle)
                    print('\n{}/20번째 숙소 정보 수집 시작'.format(k+1))
                    break
            time.sleep(3)

            house_url=driver.current_url #개별 숙소 url
            #print(house_url)
            get_information(driver)
            houseinfo=houseinfo.append(pd.DataFrame([[gu, price, rating, nrat, sh, title, housing_type, cap, bed, room, bath, res_rate, cert, clean, freeparking, laundry, camera,review_list, npho, location, house_url]],columns=houseinfo.columns),ignore_index=True)
            driver.close() #개별 숙소 탭 닫기
            driver.switch_to.window(original_window) #숙소 목록 페이지로 돌아옴
            driver.implicitly_wait(1)
        houseinfo.to_csv('C:\\Users\\ihoon\\Documents\\skk\\DSCover\\housbeinfo_ganganamgu.csv')
        print('{}page export complete'.format(j+1))
        print('\n {} 숙소 {}개 수집 완료'.format(list(url_list.keys())[i],(j+1)*20))
        driver.implicitly_wait(3) #크롤링 방지 회피용
        driver.find_element_by_xpath('//*[@id="site-content"]/div[2]/div[4]/div/div/nav/div/a[5]').click() #다음 페이지로 이동
        driver.implicitly_wait(10)
    print('\n===========================\n{} 숙소 정보 수집 완료\n==========================='.format(list(url_list.keys())[i]))
    driver.implicitly_wait(10) #크롤링 방지 회피용
    print('잠시 휴식')

print('수집 완료: {}'.format(houseinfo.shape))



강남구 숙소 정보 수집 시작


1/6번째 페이지 로드

1/20번째 숙소 정보 수집 시작

2/20번째 숙소 정보 수집 시작

3/20번째 숙소 정보 수집 시작

4/20번째 숙소 정보 수집 시작

5/20번째 숙소 정보 수집 시작

6/20번째 숙소 정보 수집 시작

7/20번째 숙소 정보 수집 시작

8/20번째 숙소 정보 수집 시작

9/20번째 숙소 정보 수집 시작

10/20번째 숙소 정보 수집 시작

11/20번째 숙소 정보 수집 시작

12/20번째 숙소 정보 수집 시작

13/20번째 숙소 정보 수집 시작

14/20번째 숙소 정보 수집 시작

15/20번째 숙소 정보 수집 시작

16/20번째 숙소 정보 수집 시작

17/20번째 숙소 정보 수집 시작

18/20번째 숙소 정보 수집 시작

19/20번째 숙소 정보 수집 시작

20/20번째 숙소 정보 수집 시작
1page export complete

 강남구 숙소 20개 수집 완료

2/6번째 페이지 로드

1/20번째 숙소 정보 수집 시작

2/20번째 숙소 정보 수집 시작

3/20번째 숙소 정보 수집 시작

4/20번째 숙소 정보 수집 시작

5/20번째 숙소 정보 수집 시작

6/20번째 숙소 정보 수집 시작

7/20번째 숙소 정보 수집 시작

8/20번째 숙소 정보 수집 시작

9/20번째 숙소 정보 수집 시작

10/20번째 숙소 정보 수집 시작

11/20번째 숙소 정보 수집 시작

12/20번째 숙소 정보 수집 시작

13/20번째 숙소 정보 수집 시작

14/20번째 숙소 정보 수집 시작

15/20번째 숙소 정보 수집 시작

16/20번째 숙소 정보 수집 시작

17/20번째 숙소 정보 수집 시작

18/20번째 숙소 정보 수집 시작

19/20번째 숙소 정보 수집 시작

20/20번째 숙소 정보 수집 시작
2page export complete

 강남구 숙소 40개 수집 완료

3/6번째 페이지 로드

1/20번째 숙소 정보 수집 시작

2/20번째 숙소 정보 수

In [39]:
#결과
houseinfo.head

In [33]:
#단일 숙소 내에서 작동 테스트할때 사용...
driver=webdriver.Chrome(path) #inititialize
driver.implicitly_wait(2)
driver.maximize_window()
url='https://www.airbnb.co.kr/rooms/662684816471147330?adults=1&children=0&infants=0&pets=0&check_in=2023-01-06&check_out=2023-01-08&source_impression_id=p3_1669272544_akliRjBajxevZ0JE'
driver.get(url)

amen_list=driver.find_element_by_xpath('//*[@id="site-content"]/div/div[1]/div[3]/div/div[1]/div/div[1]/div/div/section/div/div/div/div[1]/ol').text.split(sep=' · ')
cap=int(amen_list[0].replace('최대 인원 ','').replace('명',''))

if amen_list[1]=='원룸':
    room=0
else:
    room=float(amen_list[1].replace('침실 ','').replace('개',''))

try:
    for i in range(len(amen_list)):
        if re.search('침대',amen_list[i])==None:
            bed=0 if room==0 else 1
        else:
            bed=float(amen_list[i].replace('침대 ','').replace('개',''))
except:
    bed='NA'

try:
    for i in range(len(amen_list)):
        if re.search('욕실',amen_list[i])==None:
            bath=0
        elif amen_list[i]=='간이 욕실' :
                bath=1
        else:
                bath=float(amen_list[i].replace('단독 사용 ','').replace('공동 사용 ','').replace('욕실 ','').replace('개',''))
except:
    bath='NA'
                
print(amen_list, cap, room, bed, bath)

['최대 인원 2명', '침실 1개', '침대 1개', '욕실 1개'] 2 1.0 1 NA


In [14]:
bed

0